Setup:
- Run this in colab by changing the URL from 'github.com' to 'githubtocolab.com'. 
- Ensure that DOD_clinical_data.xlsx is in working directory

In [7]:
# Some packages
import pandas as pd
import numpy as np

These are some dictionaries that help with iterating through the classification boundaries.

In [8]:
# Clinical classification boundaries
clinical_ranges = [
(0, 25, "Normal Hearing"),
(25, 40, "Mild Hearing Loss"),
(40, 60, "Moderate Hearing Loss"),
(60, np.inf, "Severe and Profound hearing loss")
]

# Military boundaries - This models the table from: https://pmc.ncbi.nlm.nih.gov/articles/PMC10571680/table/table1-23312165231198374/
military_ranges = {
	'H0' : {
		'Better' : {'500' : 20, '1000' : 20, '2000' : 20},
		'Worse' : {'500' : 20, '1000' : 20, '2000' : 20}
	},
	'H1' : {
		'Better' : {'500' : 25, '1000' : 25, '2000' : 25},
		'Worse' : {'500' : 30, '1000' : 30, '2000' : 30}
	},
	'H2' : {
		'Better' : {'500' : 25, '1000' : 30, '2000' : 25},
		'Worse' : {'500' : 40, '1000' : 40, '2000' : 60}
	},
	'H3' : {
		'Better' : {'500' : np.inf, '1000' : np.inf, '2000' : np.inf},
		'Worse' : {'500' : np.inf, '1000' : np.inf, '2000' : np.inf}
	}
}

These are helper functions that take in the 500, 1000, and 2000hz dB HL values of a given participant and returns the classification. 

In [ ]:
# Helper function: Classifies a soldier's hearing profile based off of clinical metrics
def classify_clinical(RU500, RU1000, RU2000, LU500, LU1000, LU2000):

	# Clinical algorithm
	PTA_score = (RU500 + RU1000 + RU2000 + LU500 + LU1000 + LU2000)/6
	profile = ""
	for min, max, label in clinical_ranges:
		if min <= PTA_score <= max:
			profile = label
		if profile != "":
			break
	
	# Returns the profile and the PTA as an ordered pair
	return profile, PTA_score

# Helper function: Classifies a soldier's hearing profile based off of military metrics
def classify_military(RU500, RU1000, RU2000, LU500, LU1000, LU2000):

	# Military algorithm
	better_profile = ""
	worse_profile = ""
	profile = ""

	# Choose better ear with averages
	right_score = (RU500 + RU1000 + RU2000) / 3 
	left_score = (LU500 + LU1000 + LU2000) / 3 
	right_ear = {'500' : RU500, '1000' : RU1000, '2000' : RU2000}
	left_ear = {'500' : LU500, '1000' : LU1000, '2000' : LU2000}
	better_ear = right_ear if right_score <= left_score else left_ear
	worse_ear = left_ear if better_ear == right_ear else right_ear

	# Classify the better ear
	for label in military_ranges.keys():
		if better_ear['500'] <= military_ranges[label]['Better']['500'] and better_ear['1000'] <= military_ranges[label]['Better']['1000'] and better_ear['2000'] <= military_ranges[label]['Better']['2000']:
			better_profile = label
		if better_profile != "":
			break
	
	# Classify the worse ear
	for label in military_ranges.keys():
		if worse_ear['500'] <= military_ranges[label]['Worse']['500'] and worse_ear['1000'] <= military_ranges[label]['Worse']['1000'] and worse_ear['2000'] <= military_ranges[label]['Worse']['2000']:
			worse_profile = label
		if worse_profile != "":
			break
	
	# Pick the which ear's profile to use (if needed)
	profile = worse_profile if better_profile == worse_profile else max(worse_profile, better_profile)
	
	# Rename H0 into 'NH'
	profile = 'NH' if profile == 'H0' else profile
	better_profile = 'NH' if better_profile == 'H0' else better_profile
	worse_profile = 'NH' if worse_profile == 'H0' else worse_profile

	# Return a tuple
	return profile, better_profile, worse_profile

Here's an example of how to use these functions. The following code iterates through all participants in the 'DOD_clinical_data.xlsx' sheet, and outputs both the clinical and military classification in a separate file called 'output.xlsx'

In [10]:
# Read the file & set output path
input_path = 'DOD_clinical_data.xlsx'
output_path = 'output.xlsx'
df = pd.read_excel(input_path)

# Initialize variables
clinical_profiles = []
military_profiles = []
better_profiles = []
worse_profiles = []
ids = []
debug = []

# Iterate through all participants and classify.
for index, row in df.iterrows():
	clinical_profile, PTA_score = classify_clinical(row['RU500'], row['RU1000'], row['RU2000'], row['LU500'], row['LU1000'], row['LU2000'])
	military_profile, better_profile, worse_profile = classify_military(row['RU500'], row['RU1000'], row['RU2000'], row['LU500'], row['LU1000'], row['LU2000'])
	ids.append(row['ID'])
	clinical_profiles.append(clinical_profile)
	# military_profiles.append(military_profile) # Only if a single profile is needed
	better_profiles.append(better_profile)
	worse_profiles.append(worse_profile)
	debug.append(f"PTA = {PTA_score}, Better Ear = {better_profile}, Worse Ear = {worse_profile}") # Optional

# Output all values into a separate .xlsx file.
data = {
	'ID' : ids,
	'Clinical Profile' : clinical_profiles,
	'Military Profile (Better Ear)' : better_profiles,
	'Military Profile (Worse Ear)' : worse_profiles
}
final_df = pd.DataFrame(data)
final_df.to_excel(output_path)
